In [23]:
import logging
import sklearn.datasets as ds
import pandas as pd
import numpy as np
import math as mt

import time
import datetime

from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Ftrl
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.optimizers import Optimizer
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD

from itertools import product
from sklearn.metrics import mean_squared_error
from mulearn import FuzzyInductor
from mulearn.optimization import TensorFlowSolver
from mulearn.kernel import GaussianKernel

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [24]:
def create_dataset(name):

    #load dataset, in iris_X values, in iris_y labels 0 1 2
    iris_X, iris_y = ds.load_iris(return_X_y=True)  

    labels = ("Setosa", "Versicolor", "Virginica") 

    #dataframe with correct labels for respective values
    df = pd.DataFrame(iris_X, columns=["Sepal length", "Sepal width",
                                       "Petal length", "Petal width"])

    #associating 
    df['Class'] = iris_y
    df['Class'] = df['Class'].map(lambda c: labels[c])

    #dataset copy for labels 0 1
    selected_iris_dataset = iris_y.copy()

    #dataset selected with labels
    if(name == "Setosa"):        
        selected_iris_dataset[selected_iris_dataset != 0] = 2
        selected_iris_dataset[selected_iris_dataset == 0] = 1
        selected_iris_dataset[selected_iris_dataset == 2] = 0
    elif(name == "Versicolor"):
        selected_iris_dataset[selected_iris_dataset==2] = 0    
    elif(name == "Virginica"):
        selected_iris_dataset[selected_iris_dataset != 2] = 0
        selected_iris_dataset[selected_iris_dataset == 2] = 1


    return iris_X, selected_iris_dataset

In [25]:
def create_handler(path):   
    
    
    fhandler = logging.FileHandler(filename = path)
    
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    
    logger.addHandler(fhandler)  
    
    return fhandler

In [26]:
def main(nome, c, sigma, penalization, optimizer):  
    
    handler = create_handler(f"../../../log/Prove-6/Different-Optimizer/{nome}_c{str(c).replace('.','')}"
                             f"_sigma{str(sigma).replace('.','')}"
                             f"_penalization{str(penalization).replace('.','')}"
                             f"_{str(optimizer)}.log")
    
    #salvo i parametri
    logger.info(f"PARAMETRI DI PARTENZA: nome={nome}, c={c}, sigma={sigma}, penalization={penalization},"
                f"optimizer={optimizer}")
    
    

    iris_X, selected_iris_dataset = create_dataset(nome)
    
    n_iter = 100    

    # Gurobi solver & fit
    fi = FuzzyInductor(c=c, k=GaussianKernel(sigma=sigma))
    
    
    start = time.time()
    fi.fit(iris_X, selected_iris_dataset)
    end = time.time()
    
    # rmse gurobi
    gurobi_chis = fi.chis_
    logger.info(f"GUROBI_CHIS: {gurobi_chis}")
    logger.info(f"GUROBI_START: {start}")
    logger.info(f"GUROBI_END: {end}")
    logger.info(f"TEMPO_ESECUZIONE GUROBI(IN EPOCH): {(end-start)}")

    # TensorFlow solver
    fi = FuzzyInductor(solver=TensorFlowSolver(n_iter=n_iter, optimizer=optimizer,    
                                               penalization=penalization),c=c,k=GaussianKernel(sigma=sigma))
    
    try:
        start = time.time()
        fi.fit(iris_X, selected_iris_dataset)
        end = time.time()
    except (ModuleNotFoundError, ValueError):
        print('Tensorflow not available')


    # rmse TensorFlow
    tensorflow_chis = fi.chis_
    logger.info(f"TENSORFLOW_CHIS: {tensorflow_chis}") 
    logger.info(f"TENSORFLOW_START: {start}")
    logger.info(f"TENSORFLOW_END: {end}")
    
    total_tensorflow = end-start
    logger.info(f"TEMPO PER ITERAZIONI (N_ITER,TEMPO) TENSORFLOW: [({n_iter},{total_tensorflow})]")

    # calcolo distanza
    rmse_distance = abs(mean_squared_error(gurobi_chis, tensorflow_chis, squared=False))
    logger.info(f"RMSE_DISTANCE_CHIS: {rmse_distance}")

    #coppia n_iter, distance
    couple = [(n_iter, rmse_distance)]
    logger.info(f"COUPLE(N_ITER,DISTANCE RMSE):  {couple}")

    #incremento n
    n = 200
    
    iter_tot  = 5000
    
    # faccio ciclo fino a 5000
    while n <= iter_tot:
        
        chi_ = tensorflow_chis
        
        
        # TensorFlow solver
        fi = FuzzyInductor(solver=TensorFlowSolver(initial_values=chi_, n_iter=n_iter, 
                                                   optimizer=optimizer,
                                                   penalization=penalization),c=c,k=GaussianKernel(sigma=sigma))
        try:
            start = time.time()
            fi.fit(iris_X, selected_iris_dataset)
            end = time.time()
        except (ModuleNotFoundError, ValueError):
            print('Tensorflow not available')

        total_tensorflow += (end-start)
        
        # rmse TensorFlow
        tensorflow_chis = fi.chis_
        logger.info(f"TENSORFLOW_CHIS: {tensorflow_chis}")
        logger.info(f"TENSORFLOW_START: {start}")
        logger.info(f"TENSORFLOW_END: {end}")
        logger.info(f"TEMPO PER ITERAZIONI (N_ITER,TEMPO) TENSORFLOW: [({n},{total_tensorflow})]")
        

        # calcolo distanza
        rmse_distance = abs(mean_squared_error(gurobi_chis, tensorflow_chis, squared=False))
        logger.info(f"RMSE_DISTANCE_CHIS: {rmse_distance}")

        #coppia n_iter, distance
        couple = [(n, rmse_distance)]
        logger.info(f"COUPLE(N_ITER,DISTANCE RMSE):  {couple}")
        
        if(n == iter_tot):
            logger.info(f"TENSORFLOW_END: {end}")
            logger.info(f"TEMPO_ESECUZIONE TENSORFLOW(IN EPOCH): {total_tensorflow}")

        #incremento n_iter
        n += 100

    logger.removeHandler(handler)

In [36]:
main("Setosa",1,0.25,0.1,Adadelta(learning_rate=1e-4))

100%|██████████| 100/100 [00:19<00:00,  5.08it/s]


In [37]:
main("Setosa",1,0.25,0.1,Adagrad(learning_rate=1e-4))

100%|██████████| 100/100 [00:18<00:00,  5.42it/s]


In [38]:
main("Setosa",1,0.25,0.1,Adamax(learning_rate=1e-4))

100%|██████████| 100/100 [00:19<00:00,  5.08it/s]


In [39]:
main("Setosa",1,0.25,0.1,Ftrl(learning_rate=1e-4))

100%|██████████| 100/100 [00:26<00:00,  3.77it/s]


In [40]:
main("Setosa",1,0.25,0.1,Nadam(learning_rate=1e-4))

100%|██████████| 100/100 [00:26<00:00,  3.79it/s]


In [41]:
#main("Setosa",1,0.25,0.1,Optimizer(learning_rate=1e-4))

TypeError: Can't instantiate abstract class OptimizerV2 with abstract methods get_config

In [42]:
main("Setosa",1,0.25,0.1,RMSprop(learning_rate=1e-4))

100%|██████████| 100/100 [00:27<00:00,  3.60it/s]


In [43]:
main("Setosa",1,0.25,0.1,SGD(learning_rate=1e-4))

100%|██████████| 100/100 [00:16<00:00,  5.89it/s]


In [44]:
main("Setosa",1,0.25,0.1,Adam(learning_rate=1e-4))

100%|██████████| 100/100 [00:17<00:00,  5.84it/s]


In [45]:
main("Setosa",1,0.25,0.1,Adadelta(learning_rate=1e-3))

100%|██████████| 100/100 [00:17<00:00,  5.76it/s]


In [26]:
#lista = list(product(["Setosa","Versicolor","Virginica"],[1],[0.25],[0.1,10],[1e-4], repeat=1))
#print(lista)

[('Setosa', 1, 0.25, 0.1, 0.0001), ('Setosa', 1, 0.25, 10, 0.0001), ('Versicolor', 1, 0.25, 0.1, 0.0001), ('Versicolor', 1, 0.25, 10, 0.0001), ('Virginica', 1, 0.25, 0.1, 0.0001), ('Virginica', 1, 0.25, 10, 0.0001)]


In [27]:
#len(lista)

6

In [28]:
#for i in lista:
    #if i not in [('Setosa', 0.05, 0.1, 0.1),('Versicolor', 0.05, 0.1, 0.1),('Virginica', 0.05, 0.1, 0.1)]:
    #main(*i)

100%|██████████| 100/100 [00:18<00:00,  5.37it/s]


100%|██████████| 100/100 [00:18<00:00,  5.27it/s]
